<a href="https://colab.research.google.com/github/jstralko/lnl-machine-learning/blob/master/ch6-mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Here’s the code that loads MNIST images into X:

In [0]:
import numpy as np
import gzip
import struct
import requests
import io

def fetch_file(filename):
  r = requests.get('https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/tensorflow/MNIST_data/' + filename)
  return io.BytesIO(r.content)

def load_images(filename):
  
  # Open and unzip the file of images:
  with gzip.open(fetch_file(filename), 'rb') as f:
  
    # Read the header information into a bunch of variables:
    _ignored, n_images, columns, rows = struct.unpack('>IIII', f.read(16))
    # Read all the pixels into a NumPy array:
    all_pixels = np.frombuffer(f.read(), dtype=np.uint8)
  
    # Reshape the pixels into a matrix where each line is an image:
    return all_pixels.reshape(n_images, columns * rows)

def prepend_bias(X):
  # Insert a column of 1s in the position 0 of X.
  # (“axis=1” stands for: “insert a column, not a row”)
  return np.insert(X, 0, 1, axis=1)
  
# 60000 images, each 785 elements (1 bias + 28 * 28 pixels)
X_train = prepend_bias(load_images("train-images-idx3-ubyte.gz"))
# 10000 images, each 785 elements, with the same structure as X_train
X_test = prepend_bias(load_images("t10k-images-idx3-ubyte.gz"))

Should have training and test data. See X_train, X_test

```
(60000, 785)
(10000, 785)
```

In [27]:
print(X_train.shape)
print(X_test.shape)

#See the bias loaded
for i in range (0,5):
  print(X_train[i][0])

(60000, 785)
(10000, 785)
1
1
1
1
1


This code loads and prepares MNIST’s labels:

In [0]:
def load_labels(filename):
  # Open and unzip the file of images:
  with gzip.open(fetch_file(filename), 'rb') as f:
    # Skip the header bytes:
    f.read(8)
    
    # Read all the labels into a list:
    all_labels = f.read()
    
    # Reshape the list of labels into a one-column matrix:
    return np.frombuffer(all_labels, dtype=np.uint8).reshape(-1, 1)

def encode_fives(Y):
  encoded_Y = np.zeros_like(Y)
  n_labels = Y.shape[0]
  for i in range(n_labels):
    if Y[i] == 5: 
      encoded_Y[i][0] = 1
  return encoded_Y

# 60K labels, each with value 1 if the digit is a five, and 0 otherwise
Y_train = encode_fives(load_labels("train-labels-idx1-ubyte.gz"))
# 10000 labels, with the same encoding as Y_train
Y_test = encode_fives(load_labels("t10k-labels-idx1-ubyte.gz"))

Should be this:

```
(60000, 1)
(10000, 1)
```

In [29]:
print(Y_train.shape)
print(Y_test.shape)

oneCnt = 0;
zeroCnt = 0;
for i in range(len(Y_train)):
  if Y_train[i][0] == 1:
    oneCnt += 1
  else:
    zeroCnt += 1
    
print(oneCnt)
print(zeroCnt)

(60000, 1)
(10000, 1)
5421
54579


The fun part. First add the Logistic Regress from chapter 5:

In [0]:
#Chapter 5: page 66

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Forward propagation: moving data
# through the system
def forward(X, w):
    weighted_sum = np.matmul(X, w)
    return sigmoid(weighted_sum)

#Formerly predict()
def classify(X, w):
    return np.round(forward(X, w))

def loss(X, Y, w):
    y_hat = forward(X, w)
    first_term = Y * np.log(y_hat)
    second_term = (1 - Y) * np.log(1 - y_hat)
    return -np.average(first_term + second_term)

def gradient(X, Y, w):
    return np.matmul(X.T, (forward(X, w) - Y)) / X.shape[0]

def train(X, Y, iterations, lr):
    w = np.zeros((X.shape[1], 1))
    for i in range(iterations):
        print("Iteration %4d => Loss: %.20f" % (i, loss(X, Y, w)))
        w -= gradient(X, Y, w) * lr
    return w

def test(X, Y, w):
    total_examples = X.shape[0]
    correct_results = np.sum(classify(X, w) == Y)
    success_percent = correct_results * 100 / total_examples
    
    #print("\nSuccess: %d/%d (%.2f%%)" & (correct_results, total_examples, success_percent))
    print("\nSuccess: %d".format(100))


Add train and test the data!

In [58]:
# To bypass all the loading and preprocessing of the data.
# mnist library handles all this for you.
# import mnist as data
# w = train(data.X_train, data.Y_train, iterations=100, lr=1e-5)
# test(data.X_test, data.Y_test, w)

w = train(X_train, Y_train, iterations=1, lr=1e-5)
test(X_test, Y_test, w)

Iteration    0 => Loss: 0.69314718055994528623

Success: %d
